## Homework

In this homework, we'll deploy the dino or dragon model we trained in the
[previous homework](https://github.com/alexeygrigorev/mlbookcamp-code/blob/master/course-zoomcamp/cohorts/2022/08-deep-learning/homework.md).

Download the model from here:

https://github.com/SVizor42/ML_Zoomcamp/releases/download/dino-dragon-model/dino_dragon_10_0.899.h5



In tetminal use this > python -m wget -o model_dino_dragon.h5 https://github.com/SVizor42/ML_Zoomcamp/releases/download/dino-dragon-model/dino_dragon
_10_0.899.h5


In [1]:
# !wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/dino-dragon-model/dino_dragon_10_0.899.h5 -O model_dino_dragon.h5

In [2]:
import numpy as np

import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.11.0'

## Question 1

Now convert this model from Keras to TF-Lite format.

What's the size of the **converted** model?

* 21 Mb
* 43 Mb
* 80 Mb
* 164 Mb



answer: 43 Mb

In [3]:
model = keras.models.load_model('model_dino_dragon.h5')

In [4]:
# convert keras to TF-Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\DS_PC\AppData\Local\Temp\tmpvhi8w8zt\assets


INFO:tensorflow:Assets written to: C:\Users\DS_PC\AppData\Local\Temp\tmpvhi8w8zt\assets


In [5]:
with open('model_dino_dragon.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

In [6]:
!ls -lh

total 129M
drwxr-xr-x 1 DS_PC 197121    0 Nov 26 17:15 __pycache__
-rw-r--r-- 1 DS_PC 197121  292 Nov 26 11:44 Dockerfile
-rw-r--r-- 1 DS_PC 197121  16K Nov 28 14:50 homework_9.ipynb
-rw-r--r-- 1 DS_PC 197121 1.4K Nov 26 17:01 lambda_function.py
-rw-r--r-- 1 DS_PC 197121  86M Nov 22 18:44 model_dino_dragon.h5
-rw-r--r-- 1 DS_PC 197121  43M Nov 28 14:50 model_dino_dragon.tflite
-rw-r--r-- 1 DS_PC 197121 9.0K Nov 26 17:16 test.ipynb
-rw-r--r-- 1 DS_PC 197121  264 Nov 26 17:12 test.py


## Question 2

To be able to use this model, we need to know the index of the input and
the index of the output.

What's the output index for this model?

* 3
* 7
* 13
* 24



answer: 13

In [7]:
import tensorflow.lite as tflite

In [8]:
interpreter = tflite.Interpreter(model_path='model_dino_dragon.tflite')
interpreter.allocate_tensors()  # download wights

In [9]:
interpreter.get_input_details()

[{'name': 'serving_default_conv2d_input:0',
  'index': 0,
  'shape': array([  1, 150, 150,   3]),
  'shape_signature': array([ -1, 150, 150,   3]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [10]:
input_index = interpreter.get_input_details()[0]['index']

In [11]:
interpreter.get_output_details()

[{'name': 'StatefulPartitionedCall:0',
  'index': 13,
  'shape': array([1, 1]),
  'shape_signature': array([-1,  1]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [12]:
output_index = interpreter.get_output_details()[0]['index']

## Preparing the image

You'll need some code for downloading and resizing images. You can use
this code:


In [13]:
from io import BytesIO
from urllib import request

from PIL import Image


def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

```

For that, you'll need to have `pillow` installed:

```bash
pip install pillow
```

Let's download and resize this image:

https://upload.wikimedia.org/wikipedia/commons/thumb/d/df/Smaug_par_David_Demaret.jpg/1280px-Smaug_par_David_Demaret.jpg

Based on the previous homework, what should be the target size for the image?



target size must be 150, 150

In [14]:
url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/df/Smaug_par_David_Demaret.jpg/1280px-Smaug_par_David_Demaret.jpg"
img = download_image(url)

In [15]:
img = prepare_image(img, (150, 150))

C:\Users\DS_PC\AppData\Local\Temp\ipykernel_2720\2218159536.py:18: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  img = img.resize(target_size, Image.NEAREST)


## Question 3

Now we need to turn the image into numpy array and pre-process it.

> Tip: Check the previous homework. What was the pre-processing
> we did there?

After the pre-processing, what's the value in the first pixel, the R channel?

* 0.3353411
* 0.5529412
* 0.7458824
* 0.9654902



answer: 0.5529412

In [16]:
x = np.array(img)
X = np.array([x])
X_r = np.float32(X / 255.0)

In [17]:
X_r[0][0][0][0]

0.5529412

## Question 4

Now let's apply this model to this image. What's the output of the model?

* 0.17049132
* 0.39009996
* 0.60146114
* 0.82448614



In [18]:
answer: 0.82448614

In [19]:
interpreter.set_tensor(input_index, X_r)

In [20]:
interpreter.invoke()
interpreter.get_tensor(output_index)

array([[0.82448614]], dtype=float32)

In [21]:
preds = interpreter.get_tensor(output_index)

## Prepare the lambda code

Now you need to copy all the code into a separate python file. You will
need to use this file for the next two questions.

    '$ jupyter nbconvert --to script [YOUR_NOTEBOOK].ipynb'

Tip: you can test this file locally with `ipython` or Jupyter Notebook
by importing the file and invoking the function from this file.



## Docker

For the next two questions, we'll use a Docker image that we already
prepared. This is the Dockerfile that we used for creating the image:

```docker
FROM public.ecr.aws/lambda/python:3.9
COPY dino-vs-dragon-v2.tflite .
```

And pushed it to [`svizor42/zoomcamp-dino-dragon-lambda:v2`](https://hub.docker.com/r/svizor42/zoomcamp-dino-dragon-lambda/tags).


> Note: The image already contains a model and it's not the same model
> as the one we used for questions 1-4.



## Question 5

Download the base image `svizor42/zoomcamp-dino-dragon-lambda:v2`. You can easily make it by using [docker pull](https://docs.docker.com/engine/reference/commandline/pull/) command.

So what's the size of this base image?

* 139 Mb
* 329 Mb
* 639 Mb
* 929 Mb

You can get this information when running `docker images` - it'll be in the "SIZE" column.



In [22]:
!docker images

REPOSITORY                              TAG          IMAGE ID       CREATED       SIZE
tf-lite-lambda                          3.9-v2.6.2   17315431e723   2 days ago    735MB
dino_vs_dragon                          latest       c626b0c8bf0e   2 days ago    735MB
dino_dragon                             latest       c68bd8cf00bd   4 days ago    775MB
svizor42/zoomcamp-dino-dragon-lambda    v2           20ef58b21a05   10 days ago   639MB
zoomcamp-test                           latest       4755616a5ad0   2 weeks ago   1.1GB
cr.yandex/c626b0c8bf0e/dino_vs_dragon   latest       a8780b506fa4   3 weeks ago   77.8MB
ubuntu                                  latest       a8780b506fa4   3 weeks ago   77.8MB


answer: 639MB

## Question 6

Now let's extend this docker image, install all the required libraries
and add the code for lambda.

You don't need to include the model in the image. It's already included.
The name of the file with the model is `dino-vs-dragon-v2.tflite` and it's
in the current workdir in the image (see the Dockerfile above for the
reference).

Now run the container locally.

Score this image: https://upload.wikimedia.org/wikipedia/en/e/e9/GodzillaEncounterModel.jpg

What's the output from the model?

* 0.12
* 0.32
* 0.52
* 0.72



In [23]:
!python .\test.py

{'url': 'https://upload.wikimedia.org/wikipedia/en/e/e9/GodzillaEncounterModel.jpg'}


Traceback (most recent call last):
  File "C:\Users\DS_PC\anaconda3\lib\site-packages\urllib3\connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "C:\Users\DS_PC\anaconda3\lib\site-packages\urllib3\util\connection.py", line 95, in create_connection
    raise err
  File "C:\Users\DS_PC\anaconda3\lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] Подключение не установлено, т.к. конечный компьютер отверг запрос на подключение

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\DS_PC\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "C:\Users\DS_PC\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 398, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "C:\Users\DS_PC\anaconda3\lib\site-p

## Publishing it to AWS

Now you can deploy your model to AWS!

* Publish your image to ECR
* Create a lambda function in AWS, use the ECR image
* Give it more RAM and increase the timeout
* Test it
* Expose the lambda function using API Gateway

This is optional and not graded.



## Publishing to Docker hub

This is just for reference, this is how we published our image to Docker hub:

```bash
docker build -t zoomcamp-dino-dragon-lambda .
docker tag zoomcamp-dino-dragon-lambda:latest svizor42/zoomcamp-dino-dragon-lambda:v2
docker push svizor42/zoomcamp-dino-dragon-lambda:v2
```



## Submit the results

* Submit your results here: https://forms.gle/Pnx563ELg9jgjxHX6
* You can submit your solution multiple times. In this case, only the last submission will be used
* If your answer doesn't match options exactly, select the closest one


## Deadline

The deadline for submitting is **28 November 2022 (Monday), 23:00 CEST (Berlin time)**.

After that, the form will be closed.
